In [1]:
# Dependencies
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
# read weather csv file into dataframe
weather_file = "../Resources/weather_data_nyc_centralpark_2016.csv"
df_weather_data = pd.read_csv(weather_file)
df_weather_data.head()

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,1-1-2016,42,34,38.0,0.00,0.0,0
1,2-1-2016,40,32,36.0,0.00,0.0,0
2,3-1-2016,45,35,40.0,0.00,0.0,0
3,4-1-2016,36,14,25.0,0.00,0.0,0
4,5-1-2016,29,11,20.0,0.00,0.0,0


In [5]:
# read sales csv file into dataframe
sales_file = "../Resources/Sales_2016.csv"
df_sales_data = pd.read_csv(sales_file)
df_sales_data.head()

,DATE,TRANSACTION_TYPE,TOTAL_TRANSACTIONS,TOTAL_SALES,TOTAL_UNITS,TOTAL_COUPONS,TOTAL_RETURN_SALES,TOTAL_RETURN_UNITS
0,1/1/2016,S,"1,604","101,482.59","8,046",769,?,?
1,1/1/2016,R,107,?,?,?,"7,631.56",169
2,1/2/2016,S,"2,669","174,602.55","13,305","1,446",?,?
3,1/2/2016,R,214,?,?,?,"13,213.79",359
4,1/3/2016,S,"2,298","152,357.03","11,795","1,324",?,?


### Transform weather DataFrame

In [4]:
# replace string 'T's in data with null values
df_weather_data["precipitation"] = df_weather_data["precipitation"].replace('T', np.nan, regex=True)
df_weather_data["snow fall"] = df_weather_data["snow fall"].replace('T', np.nan, regex=True)
df_weather_data["snow depth"] = df_weather_data["snow depth"].replace('T', np.nan, regex=True)

# define function to convert dates since dates in raw data are in 2 different formats
def try_parsing_date(text):
    for fmt in ('%m/%d/%Y', '%d-%m-%Y'):
        try:
            return datetime.strptime(text, fmt).date()
        except ValueError:
            pass
    raise ValueError('no valid date format found')

# convert dates
i = 0
for date in df_weather_data["date"]:
    df_weather_data["date"][i] = try_parsing_date(date)
    i += 1

# rename column headers
df_weather_data = df_weather_data.rename(columns={"maximum temperature": "max_temp",
                                                  "minimum temperature": "min_temp",
                                                  "average temperature": "avg_temp",
                                                  "snow fall": "snow_fall",
                                                  "snow depth": "snow_depth"})

# convert object types to floats
df_weather_data["precipitation"] = df_weather_data.precipitation.astype(float)
df_weather_data["snow_fall"] = df_weather_data.snow_fall.astype(float)
df_weather_data["snow_depth"] = df_weather_data.snow_depth.astype(float)

df_weather_data.head()

C:\Users\Allen\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,date,max_temp,min_temp,avg_temp,precipitation,snow_fall,snow_depth
0,2016-01-01,42,34,38.0,0.0,0.0,0.0
1,2016-01-02,40,32,36.0,0.0,0.0,0.0
2,2016-01-03,45,35,40.0,0.0,0.0,0.0
3,2016-01-04,36,14,25.0,0.0,0.0,0.0
4,2016-01-05,29,11,20.0,0.0,0.0,0.0


In [5]:
# write df to csv
df_weather_data.to_csv("../Resources/Cleaned_Weather_Data.csv", index=False)

In [6]:
# set index
#df_weather_data.set_index("date", inplace=True)

### Transform sales DataFrame

In [19]:
# split Sales and Returns data from Sales file
sale_only_df = df_sales_data[df_sales_data["TRANSACTION_TYPE"] == "S"]
return_only_df = df_sales_data[df_sales_data["TRANSACTION_TYPE"] == "R"]

# select columns needed
sales_weather_cols = ["DATE", "TRANSACTION_TYPE", "TOTAL_TRANSACTIONS", "TOTAL_SALES", "TOTAL_UNITS", "TOTAL_COUPONS", "TOTAL_RETURN_SALES", "TOTAL_RETURN_UNITS"]
sale_new_df = sale_only_df[sales_weather_cols].copy()

# rename column header
sale_new_df = sale_new_df.rename(columns={"DATE": "date"})
sale_new_df.head()

,date,TRANSACTION_TYPE,TOTAL_TRANSACTIONS,TOTAL_SALES,TOTAL_UNITS,TOTAL_COUPONS,TOTAL_RETURN_SALES,TOTAL_RETURN_UNITS
0,1/1/2016,S,"1,604","101,482.59","8,046",769,?,?
2,1/2/2016,S,"2,669","174,602.55","13,305","1,446",?,?
4,1/3/2016,S,"2,298","152,357.03","11,795","1,324",?,?
6,1/4/2016,S,"1,967","109,492.91","9,075",947,?,?
8,1/5/2016,S,"1,796","108,041.47","8,424",662,?,?


In [18]:
# merge sales and weather
df_weather_data['date'] = df_weather_data['date'].astype('datetime64[ns]')
sale_new_df['date'] = sale_new_df['date'].astype('datetime64[ns]')

sale_weather_df = pd.merge(sale_new_df, df_weather_data, on="date")
sale_weather_df.head()

,date,TRANSACTION_TYPE,TOTAL_TRANSACTIONS,TOTAL_SALES,TOTAL_UNITS,TOTAL_COUPONS,TOTAL_RETURN_SALES,TOTAL_RETURN_UNITS,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,2016-01-01,S,"1,604","101,482.59","8,046",769,?,?,42,34,38.0,0.00,0.0,0
1,2016-01-02,S,"2,669","174,602.55","13,305","1,446",?,?,59,44,51.5,0.01,0.0,2
2,2016-01-03,S,"2,298","152,357.03","11,795","1,324",?,?,52,39,45.5,0.00,0.0,0
3,2016-01-04,S,"1,967","109,492.91","9,075",947,?,?,79,61,70.0,0.02,0.0,0
4,2016-01-05,S,"1,796","108,041.47","8,424",662,?,?,51,45,48.0,0.16,0.0,0


In [9]:
# create database connection
connection_string = "root:<insert password>@localhost/weather_data"
engine = create_engine(f'mysql://{connection_string}')

In [10]:
# confirm tables
engine.table_names()

['weather']

### Load DataFrame into SQL Database

In [12]:
# load dataframes into sql database
df_weather_data.to_sql(name='weather_2016', con=engine, if_exists='append', index=True)
sale_new_df.to_sql(name='sales_2016', con=engine, if_exists='append', index=True)
sale_weather_df.to_sql(name='weather_sales_2016', con=engine, if_exists='append', index=True)